In [23]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

from datetime import datetime
import sys
sys.path.insert(1, '/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/util_funcs/')
from pandas_colFuncs import isBtoDstlnu, whichBisSig, customMCmatching, B_ID

In [24]:
now = datetime.now()
print("time at start =", now)

time at start = 2021-10-06 09:13:30.473394


In [25]:
HTCondorRun = str(sys.argv[1])
print("HTCondorRun:",HTCondorRun)

HTCondorRun: -f


In [26]:
sub = str(sys.argv[2])
if sub.find("sub") == -1:
    sub = "sub02"
print("used sub:", sub)

used sub: sub02


In [27]:
save_data = True
save_preprocessedDataframe = True
tmp_data = True
take_subset = False
subset_size = 100000

delete_top_numFSP_evts=True

In [28]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "Dstlnu_SHR_BsX_p50cut/"
root_subdir = "SHR_dataSteering_run1/"   

root_path = nfs_path + "SHR_Hc_correctReco_BsX/" + root_subdir + sub + "/"

In [8]:
merged = "merged_"
if tmp_data:
    merged += "tmp_"

In [8]:
fileY4S = uproot.open(root_path + merged + "DXtagDstl.root:variables")
#afsPath = "/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/"
#fileY4S = uproot.open(afsPath + "DXtagDstl.root:variables")

In [9]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename = root_path + merged + "{}.root:variables".format(name)
    #filename = afsPath + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/sub02/merged_tmp_gammas.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/sub02/merged_tmp_electrons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/sub02/merged_tmp_pions.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/sub02/merged_tmp_kaons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/sub02/merged_tmp_muons.root:variables


In [10]:
df_FSPs = pd.concat(dfs)

In [11]:
df_Y4S = fileY4S.arrays(library="pd")

In [12]:
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])

46791043
1011345


In [13]:
# delete FSPs for which no Y4S file entry was found
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

In [14]:
df_Y4S = df_Y4S[(df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)]

df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

In [15]:
print(df_FSPs.shape[0])

2669913


### delete particles which occur more than ones based on uniqueParticleIdentifier

In [46]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(groupsFSPs_uniqParID.sort_values("count"))

In [ ]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("groupsFSPs_uniqParID.shape[0]:",groupsFSPs_uniqParID.shape[0])
print("df_Y4S.shape[0]:",df_Y4S.shape[0])

In [16]:
# delete particles which occur more than ones (keep first) and if possible keep the one with basf2_used==1
print("df_FSPs[basf2_used].value_counts():",df_FSPs["basf2_used"].value_counts())
df_FSPs = df_FSPs.sort_values("basf2_used",ascending=False).drop_duplicates(subset=("__event__","uniqueParticleIdentifier"), keep='first')
print("df_FSPs[basf2_used].value_counts():",df_FSPs["basf2_used"].value_counts())

df_FSPs[basf2_used].value_counts(): 0.0    1453461
1.0    1216452
Name: basf2_used, dtype: int64
df_FSPs[basf2_used].value_counts(): 1.0    1209487
0.0     727706
Name: basf2_used, dtype: int64


In [17]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])

df_FSPs.shape[0]: 1937193


## take a sample if used in notebook for faster processing

In [14]:
all_evt_nums = df_FSPs['__event__'].unique()
all_evt_nums.shape[0]

90

In [15]:
sample_evt_nums = np.random.choice(all_evt_nums, size=subset_size)
sample_evt_nums.shape[0]

100000

In [16]:
if take_subset:
    df_FSPssample = df_FSPs[df_FSPs['__event__'].isin(sample_evt_nums)]
    #df_Y4S=df_Y4Ssample
    df_FSPs=df_FSPssample

In [17]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("numEvents:",df_FSPs['__event__'].unique().shape[0])

df_FSPs.shape[0]: 4215
numEvents: 90


## save sample 

In [22]:
if take_subset:
    df_FSPs.to_csv(root_path + "df_FSPs_sample_{}_evts.csv".format(subset_size))
    #df_Y4S.to_csv(root_path + "df_Y4S_sample__evts.csv")

In [23]:
load = False
if load:
    df_FSPs = pd.read_csv(root_path + "df_FSPs_sample_{}_evts.csv".format(subset_size))
    #df_Y4S = pd.read_csv(root_path + "df_Y4S_sample10evts.csv")

## filter the wanted D*lnu and Hc isSignal==1 events

In [18]:
df_Y4S['isBtoDstlnu'] = df_Y4S.apply(isBtoDstlnu, axis=1)

In [19]:
df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)].shape[0]

37354

In [20]:
df_Y4S = df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)]

In [21]:
# delete FSPs for which no Y4S file entry is left after filer
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]
df_FSPs.shape[0]

1131187

### add cols with extra info for data prod

In [22]:
df_Y4S['Bsig_uniqParID'] = df_Y4S.apply(whichBisSig, axis=1)

In [23]:
# function to create col with the particles mother B's uniqueParticleIdentifier
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)

### print one event

In [47]:
df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)][["__event__",'Bsig_uniqParID','isBtoDstlnu','Hc_isSignalAcceptMissingGamma','Hc_mcPDG','Hc_genMotherPDG','Hc_uniqParID','Hc_genMotherID']][:10]

,__event__,Bsig_uniqParID,isBtoDstlnu,Hc_isSignalAcceptMissingGamma,Hc_mcPDG,Hc_genMotherPDG,Hc_uniqParID,Hc_genMotherID
0,22239789,83886081.0,1,1.0,421.0,413.0,100663296.0,3.0
1,22242808,83886082.0,1,1.0,421.0,413.0,100663296.0,3.0
2,22243407,83886081.0,1,1.0,-411.0,-413.0,100663296.0,6.0
3,22246415,83886081.0,1,1.0,-421.0,-423.0,100663296.0,13.0
4,14790197,83886082.0,1,1.0,-411.0,511.0,100663296.0,1.0
5,14794866,83886082.0,1,1.0,-421.0,-413.0,100663296.0,3.0
6,26364865,83886082.0,1,1.0,-431.0,-511.0,100663296.0,1.0
7,26367433,83886082.0,1,1.0,-411.0,-413.0,100663296.0,3.0
8,26369939,83886081.0,1,1.0,411.0,-511.0,100663296.0,2.0
9,19881295,83886082.0,1,1.0,-4122.0,-4222.0,100663296.0,4.0


In [26]:
df_Y4S[(df_Y4S['isBtoDstlnu'] == 1) & (df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)]['Hc_genMotherPDG'].value_counts()

-413.0       11839
 413.0       11706
-511.0       10713
 511.0       10670
-423.0        1813
 423.0        1794
 433.0         859
-433.0         843
 10431.0       338
-10431.0       330
-415.0         210
 415.0         196
-10413.0       107
 10413.0        84
 30443.0        65
 10411.0        61
-4212.0         61
 4222.0         60
 4212.0         59
 100443.0       56
-10411.0        56
 4112.0         49
-4114.0         48
-4222.0         48
 20433.0        42
 4114.0         42
-20433.0        41
-4112.0         37
 4214.0         32
-4214.0         30
 20443.0        28
 4224.0         25
-4224.0         24
 20413.0        14
-20413.0        13
 445.0           5
Name: Hc_genMotherPDG, dtype: int64

In [50]:
df_FSPs[df_FSPs["__event__"] == 22246415].sort_values('B_ID',ascending=False)[['mcPDG',
'B_ID',"basf2_used","basf2_Bsig","basf2_X",'genMothPDG_0', 'mcMother0_uniqParID', 'genMotherID_0',
'genMothPDG_1', 'mcMother1_uniqParID', 'genMotherID_1',
'genMothPDG_2', 'mcMother2_uniqParID', 'genMotherID_2',
'genMothPDG_3', 'mcMother3_uniqParID', 'genMotherID_3',
'genMothPDG_4', 'mcMother4_uniqParID', 'genMotherID_4',
'genMothPDG_5', 'mcMother5_uniqParID', 'genMotherID_5',
'genMothPDG_6', 'mcMother6_uniqParID', 'genMotherID_6',
'genMothPDG_7', 'mcMother7_uniqParID', 'genMotherID_7',
'genMothPDG_8', 'mcMother8_uniqParID', 'genMotherID_8',
'genMothPDG_9', 'mcMother9_uniqParID', 'genMotherID_9']]

,mcPDG,B_ID,basf2_used,basf2_Bsig,basf2_X,genMothPDG_0,mcMother0_uniqParID,genMotherID_0,genMothPDG_1,mcMother1_uniqParID,...,genMotherID_6,genMothPDG_7,mcMother7_uniqParID,genMotherID_7,genMothPDG_8,mcMother8_uniqParID,genMotherID_8,genMothPDG_9,mcMother9_uniqParID,genMotherID_9
245465,22.0,83886082,1.0,0.0,1.0,111.0,83886096.0,16.0,213.0,83886087.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
90510,321.0,83886082,1.0,0.0,0.0,-421.0,83886102.0,22.0,-423.0,83886093.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
286686,22.0,83886082,0.0,0.0,0.0,211.0,83886095.0,15.0,213.0,83886087.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
245464,22.0,83886082,1.0,0.0,1.0,111.0,83886096.0,16.0,213.0,83886087.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
286687,22.0,83886082,0.0,0.0,0.0,111.0,83886103.0,23.0,-423.0,83886093.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
90509,211.0,83886082,1.0,0.0,1.0,213.0,83886087.0,7.0,511.0,83886082.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1140,-211.0,83886082,1.0,0.0,0.0,-421.0,83886102.0,22.0,-423.0,83886093.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
1139,-211.0,83886082,1.0,0.0,1.0,-415.0,83886086.0,6.0,511.0,83886082.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
90511,-321.0,83886081,1.0,1.0,0.0,421.0,83886088.0,8.0,413.0,83886083.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0
112314,13.0,83886081,1.0,1.0,0.0,-511.0,83886081.0,1.0,300553.0,83886080.0,...,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0


## check if category combinations make sense

In [42]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

,basf2_used,basf2_Bsig,basf2_X,count
0,0.0,0.0,0.0,154089
1,0.0,1.0,0.0,54
2,1.0,0.0,0.0,48518
3,1.0,0.0,1.0,123958
4,1.0,1.0,0.0,90302


## delete the H_c FSPs (basf2_used==1 & basf2_Bsig==0 & basf2_X==0)

In [24]:
df_FSPs = df_FSPs[~((df_FSPs["basf2_used"]== 1) & (df_FSPs["basf2_Bsig"]== 0) & (df_FSPs["basf2_X"]== 0))]

In [ ]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

## delete delete_top_numFSP_evts

In [ ]:
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])
numEvtsBeforeCut = df_Y4S.shape[0]

if delete_top_numFSP_evts:
    delete_percentileLvl = 0.998
    print("used delete_percentileLvl:",delete_percentileLvl)
    numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
    
    print(numberFSPsEvts["count"].describe())
    
    percentile = numberFSPsEvts["count"].quantile(q=delete_percentileLvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    df_Y4S = df_Y4S[df_Y4S['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
print(df_FSPs.shape[0])
print(df_Y4S.shape[0])
print("deleted", (1 - df_Y4S.shape[0]/numEvtsBeforeCut),"% of events")
print(numberFSPsEvts["count"].describe())


## data Saving

### save dataframes on NFS

In [25]:
if HTCondorRun == "isHTCondorRun" or save_preprocessedDataframe == True:
    df_FSPs.to_csv(root_path + "df_FSPs_preProcessed_SHR.csv")
    df_Y4S.to_csv(root_path + "df_Y4S_preProcessed_SHR.csv")

### load dataframes from different subs and concat them

In [54]:
df_FSPs_final = df_FSPs
df_Y4S_final = df_Y4S

In [55]:
print(df_FSPs_final.shape[0])
print(df_Y4S_final.shape[0])

368403
14945


In [7]:
df_FSPs_list=[]
df_Y4S_list=[]
for sub in ["sub00","sub01","sub02"]:
    root_path = nfs_path + "SHR_Hc_correctReco_BsX/" + root_subdir + sub + "/"    
    df_FSPs = pd.read_csv(root_path + "df_FSPs_preProcessed_SHR.csv")
    df_FSPs_list.append(df_FSPs)
    df_Y4S = pd.read_csv(root_path + "df_Y4S_preProcessed_SHR.csv")
    df_Y4S_list.append(df_Y4S)
    
df_FSPs_final = pd.concat(df_FSPs_list)
df_Y4S_final = pd.concat(df_Y4S_list)

In [8]:
df_Y4S_final.shape[0]

132200

## keep only the ones where Hc genMotherPDG is 511

In [9]:
df_Y4S_final[(df_Y4S_final['isBtoDstlnu'] == 1) & (df_Y4S_final['Hc_isSignalAcceptMissingGamma'] == 1.0)]['Hc_genMotherPDG'].value_counts()

-413.0       29874
 413.0       29659
-511.0       26839
 511.0       26655
-423.0        4565
 423.0        4563
-433.0        2197
 433.0        2168
-10431.0       894
 10431.0       859
-415.0         562
 415.0         538
-10413.0       275
 10413.0       242
 30443.0       172
 100443.0      166
-10411.0       159
 4212.0        157
 10411.0       149
 4222.0        141
-4212.0        139
 4112.0        125
-4114.0        120
-4222.0        119
 4114.0        114
-4112.0        108
 20433.0       100
-20433.0        98
 4214.0         91
-4214.0         81
 20443.0        78
-4224.0         57
 4224.0         55
-20413.0        37
 20413.0        33
 445.0           8
-10433.0         2
 10441.0         1
Name: Hc_genMotherPDG, dtype: int64

In [10]:
df_Y4S_final = df_Y4S_final[(df_Y4S_final['Hc_genMotherPDG'] == 511.0) | (df_Y4S_final['Hc_genMotherPDG'] == -511.0)]

df_FSPs_final = df_FSPs_final[df_FSPs_final['__event__'].isin(df_Y4S_final["__event__"])]

In [11]:
print(df_FSPs_final.shape[0])
print(df_Y4S_final.shape[0])

1419934
53494


## investigate the neutrals cut

In [22]:
cuts = [0.050,0.100]

num_all_bg_neutrals = df_FSPs_final[((df_FSPs_final["PDG"]==22.) & (df_FSPs_final["B_ID"]==0.))].shape[0] 

for cut in cuts:
    num_bg_cutNeutrals = df_FSPs_final[((df_FSPs_final["p"]<cut) & (df_FSPs_final["PDG"]==22.) & (df_FSPs_final["B_ID"]==0.))].shape[0] 
    num_all_cutNeutrals = df_FSPs_final[((df_FSPs_final["p"]<cut) & (df_FSPs_final["PDG"]==22.))].shape[0]
    print("for cut p >",cut)
    print("this fraction of cut outs is bg:", round(num_bg_cutNeutrals/num_all_cutNeutrals,3))
    print("this fraction of bg is cut out:", round(num_bg_cutNeutrals/num_all_bg_neutrals,3))

for cut p > 0.05
this fraction of cut outs is bg: 0.908
this fraction of bg is cut out: 0.633
for cut p > 0.1
this fraction of cut outs is bg: 0.801
this fraction of bg is cut out: 0.857


In [21]:
cuts = [0.100] # no charged with p<0.05 GeV

num_all_bg_charged = df_FSPs_final[((df_FSPs_final["PDG"]!=22.) & (df_FSPs_final["B_ID"]==0.))].shape[0] 

for cut in cuts:
    num_bg_cutCharged = df_FSPs_final[((df_FSPs_final["p"]<cut) & (df_FSPs_final["PDG"]!=22.) & (df_FSPs_final["B_ID"]==0.))].shape[0] 
    num_all_cutCharged = df_FSPs_final[((df_FSPs_final["p"]<cut) & (df_FSPs_final["PDG"]!=22.))].shape[0]
    print("for cut p >",cut)
    print("this fraction is bg:", round(num_bg_cutCharged/num_all_cutCharged,3))
    print("this fraction of bg is cut out:", round(num_bg_cutCharged/num_all_bg_charged,3))

for cut p > 0.1
this fraction is bg: 0.054
this fraction of bg is cut out: 0.23


## cut on momenta for neutrals, p>50MeV

In [29]:
df_FSPs_final = df_FSPs_final[~((df_FSPs_final["p"]<0.050) & (df_FSPs_final["PDG"]==22.))]

In [30]:
print(df_FSPs_final.shape[0])
print(df_Y4S_final.shape[0])

1024354
53494


In [37]:
df_FSPs_final.keys()

Index(['Unnamed: 0', '__experiment__', '__run__', '__event__', '__candidate__',
       '__ncandidates__', '__weight__', 'basf2_X', 'basf2_used', 'basf2_Bsig',
       'isSignal', 'uniqueParticleIdentifier', 'mcErrors', 'mcPDG',
       'genMotherID', 'genMotherP', 'genMotherPDG', 'px', 'py', 'pz', 'pt',
       'p', 'E', 'kaonID', 'pionID', 'genMothPDG_0', 'genMothPDG_1',
       'genMothPDG_2', 'genMothPDG_3', 'genMothPDG_4', 'genMothPDG_5',
       'genMothPDG_6', 'genMothPDG_7', 'genMothPDG_8', 'genMothPDG_9',
       'genMotherID_0', 'genMotherID_1', 'genMotherID_2', 'genMotherID_3',
       'genMotherID_4', 'genMotherID_5', 'genMotherID_6', 'genMotherID_7',
       'genMotherID_8', 'genMotherID_9', 'mcMother0_uniqParID',
       'mcMother1_uniqParID', 'mcMother2_uniqParID', 'mcMother3_uniqParID',
       'mcMother4_uniqParID', 'mcMother5_uniqParID', 'mcMother6_uniqParID',
       'mcMother7_uniqParID', 'mcMother8_uniqParID', 'mcMother9_uniqParID',
       'PDG', 'B_ID', 'numFSPs'],
      dtyp

## save the concated df as it will be used for data prod

In [31]:
root_path_finalDF = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_dataSteering_run1/"

df_FSPs_final.to_csv(root_path_finalDF + "final_df_FSPs_preProcessed_SHR_gamma50cut.csv")
df_Y4S_final.to_csv(root_path_finalDF + "final_df_Y4S_preProcessed_SHR.csv")

# start of NN data creation

In [32]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()

minFSPs = numFSPs["count"].min()
maxFSPs = numFSPs["count"].max()
print("minFSPs:",minFSPs)
print("maxFSPs:",maxFSPs,'\n')
print("num Events:",numFSPs.shape[0],'\n')

df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

minFSPs: 5
maxFSPs: 58 

num Events: 53473 



In [33]:
data_dir = Path(nfs_path + "data/" + data_subdir + root_subdir)    
if save_data:
    data_dir.mkdir(parents=True, exist_ok=True)
print("Will save data to:", data_dir,'is', save_data ,'\n')

Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/data/Dstlnu_SHR_BsX_p50cut/SHR_dataSteering_run1 is True 



In [34]:
#minFSPs = 5
for num_FSPs_toData in range(minFSPs, maxFSPs+1):
    df_num_subset = df_FSPs_final.copy()
    df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]
    
        
    numEvents = df_num_subset.__event__.nunique()
    print("numEvents:",numEvents)
    print("num_FSPs_toData:",num_FSPs_toData)  
    if numEvents == 0:
        print("skipped because empty \n")
        continue

    if numEvents < 10:
        print("skipped because <10 events \n")
        continue
    
    num_features = 4
    leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
    SA_target =  np.zeros((numEvents, num_FSPs_toData))
    global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)
    
    event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["__event__"].unique()
    #print("len(event_list):",len(event_list))
    for i in range(numEvents):

        event_iter = event_list[i]

        global_tag_masterInfo = "evt" + str(event_iter)
        global_tag[i,-1] = global_tag_masterInfo
        #print("global_tag[i,-1]:",global_tag[i,-1])
        #print("i:",i,"event_iter:",event_iter)

        event_df = df_num_subset[df_num_subset.__event__ == event_iter]

        for j in range(num_FSPs_toData):
            #print("numParticle:",j)
            particle = event_df.iloc[j]

            #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
            leaves[i,j,0] = particle["px"]
            leaves[i,j,1] = particle["py"]
            leaves[i,j,2] = particle["pz"]
            leaves[i,j,3] = particle["E"]
            
            basf2_usage = "basf2_NONE"
            if particle["basf2_Bsig"] == 1.0:
                basf2_usage = "basf2_Bsig"
            elif particle["basf2_X"] == 1.0:
                basf2_usage = "basf2_X"
            elif particle["basf2_used"] == 0:
                basf2_usage = "basf2_bg"

            global_tag_Info = str((particle["mcPDG"])) 
            global_tag_Info += "_" + basf2_usage
            global_tag[i,j] = global_tag_Info

            label = -10 # error code if assignment fails
            B_tag_uniqID = -10 # error code if assignment fails     
            
            B_sig_uniqID = df_Y4S_final[df_Y4S_final["__event__"] == event_iter].iloc[0]['Bsig_uniqParID']
            if B_sig_uniqID == 83886082.0:
                B_tag_uniqID = 83886081.0
            elif B_sig_uniqID == 83886081.0:
                B_tag_uniqID = 83886082.0
            
            if particle["B_ID"] == B_tag_uniqID:
                label = 1 # particle belongs to X (MC truth)
            elif particle["B_ID"] == B_sig_uniqID:
                label = 2 # particle belongs to Bsig (MC truth)
            elif particle["B_ID"] == 0:
                label = 0 # background (MC truth)
            
            
            SA_target[i,j] = label
            
        del event_df
        
        
    # shuffle the data    
    for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
        perms = np.random.permutation(leaves.shape[1])

        leaves[idx,:] = leaves[idx,perms]
        SA_target[idx,:] = SA_target[idx,perms]
        global_tag[idx,0:-1] = global_tag[idx,perms]
        
        
         


    #print(global_tag)
    train_ratio = 0.82
    validation_ratio = 0.12
    test_ratio = 0.06

    print("leaves.shape:",leaves.shape)
    print("SA_target.shape:",SA_target.shape)
    print("global_tag.shape:",global_tag.shape)


    print("leaves[0]:",leaves[0])
    print("SA_target[0]:",SA_target[0])
    print("global_tag[0]:",global_tag[0])

    x=leaves
    y=SA_target
    z=global_tag

    x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
    x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

    if save_data==True:
        np.save(data_dir / "leaves_train_FSP{}.npy".format(num_FSPs_toData), x_train)
        np.save(data_dir / "is_left_arr_train_FSP{}.npy".format(num_FSPs_toData), y_train)
        np.save(data_dir / "global_tag_train_FSP{}.npy".format(num_FSPs_toData), z_train)

        np.save(data_dir / "leaves_val_FSP{}.npy".format(num_FSPs_toData), x_val)
        np.save(data_dir / "is_left_arr_val_FSP{}.npy".format(num_FSPs_toData), y_val)
        np.save(data_dir / "global_tag_val_FSP{}.npy".format(num_FSPs_toData), z_val)

        np.save(data_dir / "leaves_test_FSP{}.npy".format(num_FSPs_toData), x_test)
        np.save(data_dir / "is_left_arr_test_FSP{}.npy".format(num_FSPs_toData), y_test)
        np.save(data_dir / "global_tag_test_FSP{}.npy".format(num_FSPs_toData), z_test)

    
    print("")
    #del df_num_subset


    del df_num_subset
                                          

numEvents: 1
num_FSPs_toData: 5
skipped because <10 events 

numEvents: 13
num_FSPs_toData: 6
leaves.shape: (13, 6, 4)
SA_target.shape: (13, 6)
global_tag.shape: (13, 7)
leaves[0]: [[-1.31769907 -1.71121681  0.46163255  2.20855126]
 [-0.05611247  0.04838415  0.24944316  0.29528193]
 [-0.99028981  0.24283016  2.10930467  2.39426889]
 [ 0.70648092  0.05610949 -0.93426108  1.17740011]
 [ 0.0623211   0.0740999   0.00421407  0.09691477]
 [ 0.6453467   0.20104177  0.44341314  0.82035679]]
SA_target[0]: [2. 1. 1. 1. 0. 1.]
global_tag[0]: [b'11.0_basf2_X' b'-211.0_basf2_Bsig' b'321.0_basf2_Bsig'
 b'-13.0_basf2_Bsig' b'nan_basf2_X' b'-211.0_basf2_Bsig' b'evt17226631']

numEvents: 61
num_FSPs_toData: 7
leaves.shape: (61, 7, 4)
SA_target.shape: (61, 7)
global_tag.shape: (61, 8)
leaves[0]: [[ 0.4164198   0.53697294 -0.25454798  0.73893163]
 [ 0.06195676 -0.07559867  0.06243305  0.1814706 ]
 [-0.02544531  0.00601237 -0.04855982  0.05515132]
 [ 0.47449929 -1.11988652  0.57082099  1.43138009]
 [-0.05


numEvents: 3317
num_FSPs_toData: 15
leaves.shape: (3317, 15, 4)
SA_target.shape: (3317, 15)
global_tag.shape: (3317, 16)
leaves[0]: [[ 0.00466241 -0.13291293  0.15058397  0.20090574]
 [ 0.10824957 -0.0358768  -0.22633563  0.25344217]
 [ 0.19974931 -1.12455428  1.38345385  1.79400862]
 [ 0.04218119 -0.01441183 -0.0345133   0.05637483]
 [ 0.12557989  0.57817471  0.89600551  1.0827567 ]
 [ 0.03560304 -0.00404638 -0.04858678  0.06037072]
 [-0.13967767 -0.18000318  0.3243241   0.39635479]
 [-0.06759957  0.08180291 -0.15409993  0.1871048 ]
 [ 0.04294223  0.00917208  0.10359541  0.1792765 ]
 [ 0.01496936 -0.03280291  0.06488185  0.07422781]
 [-0.14681837 -0.1762608   0.52888113  0.59314302]
 [ 1.03546178  0.04787748  0.54918754  1.27271257]
 [ 0.08974034 -0.16862315  0.34181836  0.39156977]
 [-0.05448128  0.07145701 -0.08437508  0.12326179]
 [-0.60773492  0.70990992 -0.8818953   1.28493323]]
SA_target[0]: [1. 0. 1. 2. 2. 0. 2. 1. 2. 0. 2. 1. 1. 0. 2.]
global_tag[0]: [b'22.0_basf2_X' b'nan_ba


numEvents: 4212
num_FSPs_toData: 21
leaves.shape: (4212, 21, 4)
SA_target.shape: (4212, 21)
global_tag.shape: (4212, 22)
leaves[0]: [[ 3.32391448e-02  6.03660867e-02  8.08163732e-03  6.93845662e-02]
 [ 8.99787992e-03  2.61244085e-02 -5.10977991e-02  5.80898583e-02]
 [ 2.76496857e-01  6.12815283e-02  4.57687676e-02  2.86881017e-01]
 [ 3.28588396e-01 -2.65819617e-02 -9.57988873e-02  3.43299231e-01]
 [ 5.42437956e-02 -1.64470263e-03  3.99843641e-02  6.74080394e-02]
 [-2.12755855e-02 -5.53924069e-02  8.68953243e-02  1.05222463e-01]
 [ 3.18514317e-01  2.00030595e-01 -2.50297487e-02  3.76948402e-01]
 [ 1.39818683e-01  3.15555409e-02 -1.87227666e-01  2.74005671e-01]
 [ 1.81859469e+00  4.08196956e-01  3.08024049e-01  1.88912427e+00]
 [-6.90006688e-02  7.51239359e-02  3.03306319e-02  1.06417317e-01]
 [ 2.55783424e-02  9.33962762e-02 -7.34682903e-02  1.85079978e-01]
 [ 8.55196714e-02 -1.93983823e-01  3.24398696e-01  3.87527873e-01]
 [ 2.17266217e-01 -8.31290409e-02 -5.08761853e-02  2.76013270e-


numEvents: 1411
num_FSPs_toData: 26
leaves.shape: (1411, 26, 4)
SA_target.shape: (1411, 26)
global_tag.shape: (1411, 27)
leaves[0]: [[ 0.03301015 -0.15627216  0.07513804  0.52428366]
 [-0.45337915 -0.30740806  0.70761925  0.90568056]
 [-0.18394852  0.0243372   0.13564694  0.26890396]
 [ 0.09931546  0.07170989  0.03518144  0.12745039]
 [-0.0203504   0.20100571  0.12414937  0.23712972]
 [-0.30484682  0.09291729  0.10449723  0.33538766]
 [-0.03570799 -0.06522986  0.0284225   0.07961051]
 [ 0.8726663  -0.81456435 -1.02655661  1.57798723]
 [-0.04359913  0.0836326   0.26514465  0.31412875]
 [ 0.1500386   0.00195167  0.15065667  0.21263307]
 [ 0.12872295  0.39668256  0.24117987  0.6897908 ]
 [ 0.0201551   0.1092215   0.34438667  0.38783712]
 [-0.02830565  0.01425709  0.06249715  0.07007402]
 [ 0.06241724  0.09378646  0.10225299  0.15214298]
 [-0.06818922 -0.0822276   0.03638441  0.11284933]
 [-0.01216264  0.04049018 -0.03393434  0.05421184]
 [ 0.12695783 -0.17226115  0.03563413  0.25795713]



numEvents: 285
num_FSPs_toData: 30
leaves.shape: (285, 30, 4)
SA_target.shape: (285, 30)
global_tag.shape: (285, 31)
leaves[0]: [[-0.06136363  0.3845441  -0.25840759  0.46734799]
 [-0.20777735 -1.7646625   0.8833667   1.98713502]
 [-0.05957104 -0.13124917  0.02699016  0.1466408 ]
 [-0.12453666 -0.04624359 -0.02599264  0.1353642 ]
 [ 0.48478505  0.15894844  0.10926159  0.52174634]
 [ 0.04455537  0.03296369 -0.01015087  0.05634559]
 [-0.05460386 -0.11332155  0.04862192  0.19408075]
 [ 0.12711495 -0.01168642  0.06469324  0.1998997 ]
 [ 0.08703588 -0.165915    0.31838879  0.39491055]
 [ 0.01857657 -0.05384552 -0.00839021  0.05757451]
 [ 0.12823527 -0.04082881  0.10588046  0.17123652]
 [ 0.01915684  0.09940343 -0.10130777  0.14321763]
 [ 0.2499972  -0.11077744  0.01631266  0.27392762]
 [ 0.85189074  1.55983531 -0.17240264  1.78564469]
 [-0.04941621  0.07297378 -0.05602651  0.1044323 ]
 [-0.12823133 -0.07621703 -0.12917075  0.24169668]
 [-0.10351557 -0.11767761 -0.01783305  0.21062147]
 [ 0


numEvents: 77
num_FSPs_toData: 34
leaves.shape: (77, 34, 4)
SA_target.shape: (77, 34)
global_tag.shape: (77, 35)
leaves[0]: [[ 1.32925153e+00  3.78089547e-01  1.17588890e+00  1.81454575e+00]
 [-3.68195958e-02 -8.91718194e-02  2.91126370e-01  3.36959570e-01]
 [ 7.42343888e-02 -7.45802224e-02  9.21190530e-02  1.39853044e-01]
 [-1.90358490e-01 -1.56142324e-01 -1.74192145e-01  3.01595231e-01]
 [-6.87793195e-02 -4.57485244e-02  6.02463819e-02  1.02240642e-01]
 [-9.71040353e-02  5.90384007e-02 -5.84922321e-02  1.27812627e-01]
 [-1.06058039e-01 -8.02172497e-02  2.91276854e-02  1.36130587e-01]
 [ 6.24177277e-01  5.19666255e-01 -1.26907006e-01  8.33807873e-01]
 [ 1.47922397e-01 -1.43844321e-01  5.13678849e-01  5.53568590e-01]
 [ 1.09140587e-04 -2.84156315e-02 -4.47784364e-02  5.30336534e-02]
 [ 3.54504406e-01  7.00901449e-02  3.48099828e-01  7.03900890e-01]
 [-3.61043781e-01 -1.54874146e-01 -4.29033004e-02  3.95195276e-01]
 [-3.82859170e-01 -5.99560477e-02  2.47379214e-01  4.80470852e-01]
 [-2


numEvents: 54
num_FSPs_toData: 37
leaves.shape: (54, 37, 4)
SA_target.shape: (54, 37)
global_tag.shape: (54, 38)
leaves[0]: [[ 0.02024099 -0.01098928  0.05270765  0.05752007]
 [-0.04613536 -0.02956976 -0.08983096  0.10522568]
 [ 0.12974007  0.10108053  0.16190444  0.26970855]
 [-0.05527157  0.05322896  0.01202446  0.07767146]
 [-0.12679246 -0.21559031 -0.08850677  0.29978134]
 [-0.06090349 -0.05321548 -0.0138228   0.08204993]
 [-0.0303491  -0.06671333 -0.02136042  0.07634137]
 [-0.00500463  0.10527173  0.08216866  0.51144486]
 [-0.09180086  0.04369305  0.02945926  0.1058505 ]
 [-1.37689567  0.75760192  1.39633989  2.1049302 ]
 [-0.02761393  0.01022276 -0.04130447  0.05072567]
 [-0.1653035   0.00653097  0.11278403  0.20022022]
 [ 0.12425198 -0.40658611  0.48131192  0.65718481]
 [ 0.35981357 -0.55831498 -0.22202159  0.85685003]
 [ 0.07663164 -0.20798512 -0.15646641  0.27131523]
 [-0.16341762  1.59280121  0.17898907  1.61113574]
 [ 0.03180321 -0.21419047  0.09081394  0.23481093]
 [ 0.066

numEvents: 34
num_FSPs_toData: 40
leaves.shape: (34, 40, 4)
SA_target.shape: (34, 40)
global_tag.shape: (34, 41)
leaves[0]: [[-7.23865256e-02 -1.22677661e-01  1.38887931e-02  1.43117142e-01]
 [-1.01898484e-01  3.89271155e-02  7.56070912e-02  1.32721715e-01]
 [ 9.71346200e-02  1.19226329e-01  6.12467229e-02  1.65533118e-01]
 [ 7.26614147e-02  7.65158609e-02  3.49731743e-02  1.11164208e-01]
 [ 4.73947674e-02 -3.91063280e-02  1.41506428e-02  6.30540210e-02]
 [-2.96981722e-01 -1.51715505e+00 -3.66914570e-01  1.58889400e+00]
 [ 1.66243494e-01 -1.71562389e-01  5.96544743e-02  2.46229991e-01]
 [ 3.31272095e-01 -1.25980094e-01 -5.50282821e-02  3.84863859e-01]
 [ 1.81010976e-01  4.37092602e-01 -5.16721271e-02  4.95948402e-01]
 [ 2.76210725e-01  2.05918455e+00  1.33390594e+00  2.46897523e+00]
 [-2.65709553e-02 -5.84323332e-03 -5.36046177e-02  6.01133436e-02]
 [ 9.21438336e-02  1.00892730e-01  8.56075622e-03  1.36905499e-01]
 [ 2.78828368e-02  1.03297569e-01 -3.11711859e-02  1.11442735e-01]
 [ 1.

numEvents: 11
num_FSPs_toData: 43
leaves.shape: (11, 43, 4)
SA_target.shape: (11, 43)
global_tag.shape: (11, 44)
leaves[0]: [[ 7.43751973e-02 -6.01162377e-04  1.13662124e-01  1.94758831e-01]
 [-3.49708870e-02  3.17971706e-02  3.96369025e-02  6.16855497e-02]
 [ 1.80405716e-03 -1.61737397e-01  2.37211928e-01  3.19236012e-01]
 [-2.47966293e-02 -2.23168358e-02  4.55235466e-02  5.64385266e-02]
 [ 3.91698211e-01 -5.85520342e-02  1.58331379e-01  4.26526266e-01]
 [ 4.82766539e-01 -1.10376336e-01 -4.86709982e-01  7.08246402e-01]
 [ 1.17300995e-01  3.32510948e-01  2.58980215e-01  4.59209863e-01]
 [ 1.89677522e-01  2.13043299e-02  1.11027509e-02  5.29406912e-01]
 [-6.65812269e-02  1.03021972e-01  1.63626239e-01  2.04499713e-01]
 [-1.86397359e-02  2.17520613e-02 -4.88504730e-02  5.66300330e-02]
 [ 7.25059658e-02 -4.06563357e-02  2.17501789e-01  2.71471865e-01]
 [-1.98696554e-02  3.12630869e-02  6.21698946e-02  7.23690514e-02]
 [-7.09894374e-02 -8.43608141e-01  1.24720531e-02  9.80095260e-01]
 [ 2.

In [35]:
print("saving is done")
now = datetime.now()
print("time at end =", now)

saving is done
time at end = 2021-10-06 09:44:36.504615


In [ ]:
#why only one FSP in so many events?

In [59]:
df_num_subset = df_FSPs_final.copy()
df_num_subset = df_num_subset[df_num_subset['numFSPs'] == 1]

In [51]:
df_FSPs_final.keys()

Index(['Unnamed: 0', '__experiment__', '__run__', '__event__', '__candidate__',
       '__ncandidates__', '__weight__', 'basf2_X', 'basf2_used', 'basf2_Bsig',
       'isSignal', 'uniqueParticleIdentifier', 'mcErrors', 'mcPDG',
       'genMotherID', 'genMotherP', 'genMotherPDG', 'px', 'py', 'pz', 'pt',
       'p', 'E', 'kaonID', 'pionID', 'genMothPDG_0', 'genMothPDG_1',
       'genMothPDG_2', 'genMothPDG_3', 'genMothPDG_4', 'genMothPDG_5',
       'genMothPDG_6', 'genMothPDG_7', 'genMothPDG_8', 'genMothPDG_9',
       'genMotherID_0', 'genMotherID_1', 'genMotherID_2', 'genMotherID_3',
       'genMotherID_4', 'genMotherID_5', 'genMotherID_6', 'genMotherID_7',
       'genMotherID_8', 'genMotherID_9', 'mcMother0_uniqParID',
       'mcMother1_uniqParID', 'mcMother2_uniqParID', 'mcMother3_uniqParID',
       'mcMother4_uniqParID', 'mcMother5_uniqParID', 'mcMother6_uniqParID',
       'mcMother7_uniqParID', 'mcMother8_uniqParID', 'mcMother9_uniqParID',
       'PDG', 'B_ID', 'numFSPs'],
      dtyp

In [52]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

,basf2_used,basf2_Bsig,basf2_X,count
0,0.0,0.0,0.0,591463
1,0.0,1.0,0.0,1
2,1.0,0.0,1.0,479138
3,1.0,1.0,0.0,349332


In [3]:
fileY4S = uproot.open("/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_wChargeLessFiles_run1/DXtagDstl.root:variables")
df_Y4S = fileY4S.arrays(library="pd")

In [4]:
pions = uproot.open("/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/SHR_Hc_correctReco_BsX/SHR_wChargeLessFiles_run1/pions.root:variables")
df_pions = pions.arrays(library="pd")

NameError: name 'event_df' is not defined

In [5]:
df_Y4S.shape[0]

11

In [6]:
df_Y4S[(df_Y4S['Hc_isSignalAcceptMissingGamma'] == 1.0)].shape[0]

11